<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/train_lama_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import shutil
import glob

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# train_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/train_png.zip'
val_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
eval_zip = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
dataset_dir = '/content/lama/my_dataset/'

In [3]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil


> Cloning the repo
Cloning into 'lama'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 283 (delta 73), reused 264 (delta 66), pack-reused 0
Receiving objects: 100% (283/283), 6.49 MiB | 19.56 MiB/s, done.
Resolving deltas: 100% (73/73), done.

> Install dependencies
     |████████████████████████████████| 12.5 MB 5.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 72 kB 793 kB/s 
     |████████████████████████████████| 144 kB 65.6 MB/s 
     |████████████████████████████████| 841 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 57.1 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 948 kB 59.3 MB/s 
     |████████████████████████████████| 47.7 MB 68 kB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
     |████████████████████████████████| 112 kB 85.7

In [4]:
# copy the training set over
os.mkdir(dataset_dir)
shutil.copy(val_zip, dataset_dir + 'val.zip')
shutil.unpack_archive(dataset_dir + 'val.zip', dataset_dir + '/val/')

In [10]:
# makes a copy for eval and visual test
shutil.copytree(dataset_dir + 'val_source/', dataset_dir + 'eval_source/')
shutil.copytree(dataset_dir + 'val_source/', dataset_dir + 'visual_test_source/')

'/content/my_dataset/visual_test_source/'

In [8]:
# copies images to the training folder
img_list = glob.glob(dataset_dir + '/val/*')
print(f'Number of images: {len(img_list)}')
img_list = [f for f in img_list if 'mask' not in f]
print(f'Number of images: {len(img_list)}')

os.mkdir(dataset_dir + 'train/')
for i in range(2100):
    shutil.copy(img_list[i], dataset_dir + 'train/' + os.path.basename(img_list[i]))


Number of images: 122852
Number of images: 61426


In [17]:
!touch my_dataset.yaml
!echo "data_root_dir: $(pwd)/my_dataset/" >> my_dataset.yaml
!echo "out_root_dir: $(pwd)/experiments/" >> my_dataset.yaml
!echo "tb_dir: $(pwd)/tb_logs/" >> my_dataset.yaml
!mv my_dataset.yaml ${PWD}/configs/training/location/

In [18]:
!export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 bin/train.py -cn lama-fourier location=my_dataset data.batch_size=10
# !cat ${PWD}/configs/training/data/abl-04-256-mh-dist

Detectron v2 is not installed
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'hydra/overrides': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'trainer/any_gpu_large_ssim_ddp_final': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:127: UserWarning: In 'evaluator/default_inpainted': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  See {url} for more information"""
/usr